In [8]:
import requests
import numpy as np
import pandas as pd

In [2]:
uni_grads_url = 'https://banana.datausa.io/api/data?measures=Completions&drilldowns=PUMA,University'
eng_grads_url = 'http://banana.datausa.io/api/data?CIP=14&drilldowns=PUMA&measure=Completions'
uni_pop_url = 'https://banana.datausa.io/api/data?measure=ygcpop%20RCA,Total%20Population,Total%20Population%20MOE%20Appx,Average%20Wage,Average%20Wage%20Appx%20MOE,Record%20Count&drilldowns=PUMA&Record%20Count%3E=5'
eng_pop_url = 'https://banana.datausa.io/api/data?CIP2=14&measure=ygcpop%20RCA,Total%20Population,Total%20Population%20MOE%20Appx,Average%20Wage,Average%20Wage%20Appx%20MOE,Record%20Count&drilldowns=PUMA&Record%20Count%3E=5&Workforce%20Status=true'

### Using this data, please write a script that creates a new table that shows the following metrics for each State by year:
- total number of graduations
- total number of graduations from engineering majors
- % of each states graduations that were from engineering majors 
- Estimate of total wages for the entire population 
- Estimate of total wages for all engineers 
- % of wages that were from engineers 
- Any additional columns that may be helpful from a Data Engineering perspective (think items that could help with internal tasks or maintenance, no need to add columns for better analysis)


In [27]:
## Total number of graduations

# Get the data
r_uni_grads = requests.get(uni_grads_url)
uni_grads_data = r_uni_grads.json()

# Create dataframe
uni_grads_df = pd.DataFrame(uni_grads_data['data'])

# Cleanup column names
uni_grads_df.columns= uni_grads_df.columns.str.lower()
uni_grads_df.columns= uni_grads_df.columns.str.replace(' ', '_')
#uni_grads_df.head()

# Add state column from last two characters of PUMA
state_parser = lambda x: x[-2:]
uni_grads_df['state'] = uni_grads_df['puma'].apply(state_parser)

# Fill in missing states
uni_grads_df.loc[(uni_grads_df.state == ''),'state'] = 'ZZ' # unknown

# Add state-year column
uni_grads_df['state_year'] = uni_grads_df['state'] + '-' + uni_grads_df['year']

# Sum completions by state
total_graduations_per_state_by_year = uni_grads_df.groupby('state_year')['completions'].sum()

In [82]:
total_graduations_per_state_by_year

state_year
AK-2012    5021
AK-2013    5332
AK-2014    5781
AK-2015    5690
AK-2016    5630
           ... 
ZZ-2013     999
ZZ-2014     977
ZZ-2015    1053
ZZ-2016    1033
ZZ-2017    1067
Name: completions, Length: 330, dtype: int64

In [31]:
## Total number of engineering graduations

# Get the data
r_eng_grads = requests.get(eng_grads_url)
eng_grads_data = r_eng_grads.json()

# Create dataframe
eng_grads_df = pd.DataFrame(eng_grads_data['data'])

# Cleanup column names
eng_grads_df.columns= eng_grads_df.columns.str.lower()
eng_grads_df.columns= eng_grads_df.columns.str.replace(' ', '_')

# Add state column from last two characters of PUMA
state_parser = lambda x: x[-2:]
eng_grads_df['state'] = eng_grads_df['puma'].apply(state_parser)

# Fill in missing states
eng_grads_df.loc[(eng_grads_df.state == ''),'state'] = 'ZZ' # unknown

# Add state-year column
eng_grads_df['state_year'] = eng_grads_df['state'] + '-' + eng_grads_df['year']

# Sum completions by state
engrg_graduations_per_state_by_year = eng_grads_df.groupby('state_year')['completions'].sum()

In [83]:
engrg_graduations_per_state_by_year

state_year
AK-2012    223
AK-2013    228
AK-2014    248
AK-2015    243
AK-2016    239
          ... 
ZZ-2013      0
ZZ-2014      0
ZZ-2015      0
ZZ-2016      0
ZZ-2017      4
Name: completions, Length: 318, dtype: int64

In [78]:
# Join graduations
df = pd.concat([total_graduations_per_state_by_year, engrg_graduations_per_state_by_year], axis=1)
df.columns = ['total_graduations', 'engrg_graduations']

# Calculate percentage of engrg degrees
df['engrg_grad_percent'] = df['engrg_graduations']/df['total_graduations']

In [79]:
df

,total_graduations,engrg_graduations,engrg_grad_percent
AK-2012,5021,223.0,0.044413
AK-2013,5332,228.0,0.042761
AK-2014,5781,248.0,0.042899
AK-2015,5690,243.0,0.042707
AK-2016,5630,239.0,0.042451
...,...,...,...
ZZ-2013,999,0.0,0.000000
ZZ-2014,977,0.0,0.000000
ZZ-2015,1053,0.0,0.000000
ZZ-2016,1033,0.0,0.000000


In [56]:
## Total wages

# Get the data
r_uni_pop = requests.get(uni_pop_url)
uni_pop_data = r_uni_pop.json()

# Create dataframe
uni_pop_df = pd.DataFrame(uni_pop_data['data'])

# Cleanup column names
uni_pop_df.columns= uni_pop_df.columns.str.lower()
uni_pop_df.columns= uni_pop_df.columns.str.replace(' ', '_')

# Add state column from last two characters of PUMA
state_parser = lambda x: x[-2:].upper()
uni_pop_df['state'] = uni_pop_df['slug_puma'].apply(state_parser)

# Fill in missing states
uni_pop_df.loc[(uni_pop_df.state == ''),'state'] = 'ZZ' # unknown

# Add state-year column
uni_pop_df['state_year'] = uni_pop_df['state'] + '-' + uni_pop_df['year']

# Estimate total wage
uni_pop_df['total_wage'] = uni_pop_df['total_population'] * uni_pop_df['average_wage']

# Sum total wage by state
total_wages_per_state_by_year = uni_pop_df.groupby('state_year')['total_wage'].sum()

In [57]:
total_wages_per_state_by_year

state_year
02-2014    2.634888e+09
02-2015    2.694568e+09
02-2016    2.463861e+09
02-2017    2.696661e+09
02-2018    3.083047e+09
               ...     
WY-2014    1.324791e+10
WY-2015    1.375432e+10
WY-2016    1.256858e+10
WY-2017    1.296584e+10
WY-2018    1.405637e+10
Name: total_wage, Length: 315, dtype: float64

In [58]:
## Engineering wages

# Get the data
r_eng_pop = requests.get(eng_pop_url)
eng_pop_data = r_eng_pop.json()

# Create dataframe
eng_pop_df = pd.DataFrame(eng_pop_data['data'])

# Cleanup column names
eng_pop_df.columns= eng_pop_df.columns.str.lower()
eng_pop_df.columns= eng_pop_df.columns.str.replace(' ', '_')

# Add state column from last two characters of PUMA
state_parser = lambda x: x[-2:].upper()
eng_pop_df['state'] = eng_pop_df['slug_puma'].apply(state_parser)

# Fill in missing states
eng_pop_df.loc[(eng_pop_df.state == ''),'state'] = 'ZZ' # unknown

# Add state-year column
eng_pop_df['state_year'] = eng_pop_df['state'] + '-' + eng_pop_df['year']

# Estimate total wage
eng_pop_df['total_wage'] = eng_pop_df['total_population'] * eng_pop_df['average_wage']

# Sum total wage by state
engrg_wages_per_state_by_year = eng_pop_df.groupby('state_year')['total_wage'].sum()

In [59]:
engrg_wages_per_state_by_year

state_year
02-2014    107049356.0
02-2015    103998287.0
02-2016     83033312.0
02-2017    102679163.0
02-2018     95330807.0
              ...     
WY-2014    446898816.0
WY-2015    332522076.0
WY-2016    418736244.0
WY-2017    308657656.0
WY-2018    454706813.0
Name: total_wage, Length: 299, dtype: float64

In [80]:
# Join wages
df2 = pd.concat([total_wages_per_state_by_year, engrg_wages_per_state_by_year], axis=1)
df2.columns = ['total_wages', 'engrg_wages']

# Calculate percentage of engrg wages
df2['engrg_wage_percent'] = df2['engrg_wages']/df2['total_wages']

In [71]:
df2

,total_wages,engrg_wages,engrg_wage_percent
02-2014,2.634888e+09,107049356.0,0.040628
02-2015,2.694568e+09,103998287.0,0.038596
02-2016,2.463861e+09,83033312.0,0.033700
02-2017,2.696661e+09,102679163.0,0.038076
02-2018,3.083047e+09,95330807.0,0.030921
...,...,...,...
WY-2014,1.324791e+10,446898816.0,0.033734
WY-2015,1.375432e+10,332522076.0,0.024176
WY-2016,1.256858e+10,418736244.0,0.033316
WY-2017,1.296584e+10,308657656.0,0.023805


In [76]:
# Combine gradudation and wage dataframes
final_df = pd.concat([df, df2], axis=1).reset_index().rename(columns={final_df.index.name:'state_year'})

# Add timestamp
final_df['processed_at'] = pd.to_datetime("today")

In [84]:
final_df

# could export if we wanted
# final_df.to_csv('summersalt_exercise_result.csv', index=False)

,index,total_graduations,engrg_graduations,engrg_grad_percent,total_wages,engrg_wages,engrg_wage_percent,processed_at
0,AK-2012,5021.0,223.0,0.044413,NaN,NaN,NaN,2020-09-09 14:42:09.477053
1,AK-2013,5332.0,228.0,0.042761,NaN,NaN,NaN,2020-09-09 14:42:09.477053
2,AK-2014,5781.0,248.0,0.042899,1.912699e+10,5.486463e+08,0.028684,2020-09-09 14:42:09.477053
3,AK-2015,5690.0,243.0,0.042707,1.847590e+10,9.003554e+08,0.048731,2020-09-09 14:42:09.477053
4,AK-2016,5630.0,239.0,0.042451,1.938393e+10,7.467804e+08,0.038526,2020-09-09 14:42:09.477053
...,...,...,...,...,...,...,...,...
432,VT-2018,NaN,NaN,NaN,1.506539e+10,6.590172e+08,0.043744,2020-09-09 14:42:09.477053
433,WA-2018,NaN,NaN,NaN,2.228773e+11,1.656878e+10,0.074340,2020-09-09 14:42:09.477053
434,WI-2018,NaN,NaN,NaN,1.433327e+11,5.825935e+09,0.040646,2020-09-09 14:42:09.477053
435,WV-2018,NaN,NaN,NaN,3.251834e+10,1.155093e+09,0.035521,2020-09-09 14:42:09.477053


### A couple of thoughts

1. For sake of speed, I have valiated DRY throughout. I'd update this code by creating two classes (Degree, Wage) to house the logic for getting and shaping the data. This would also be more extensible for other degree types, etc.
2. In this example, I'd want to build up a PUMA to state reference table that could handle the edge cases as presented with the wage datasets.